## generate erdos testing dataset for TELGEN

In [1]:
from solver.linprog import linprog
from tqdm import tqdm

import gzip
import pickle
import torch
from scipy.linalg import LinAlgWarning
from scipy.optimize._optimize import OptimizeWarning
# from scipy.optimize import OptimizeWarning
import warnings
import numpy as np
from functools import partial
import random
import pickle 

In [2]:
root = 'raw/'

### Resource Allocation

#### input: for one graph   
#### G(V, E, c): random graph (strongly connected)
#### (s, t, d) \in [S, T, D]  
#### for every (s, t, d), there is a set p \in Pd (k-shortest path algorithm (4/5/6))

In [3]:
import networkx as nx
import matplotlib.pyplot as plt

### generate and save connected and directed ER graph different nodes and p
### generate capacities for these graphs

In [5]:
np.random.seed(2024)

def generate_random_capacities(graph):
    for u, v in graph.edges():
        # Generate a random capacity for the edge (u, v)
        capacity = random.uniform(1000, 5000)  # Adjust the range as needed
        # Assign the capacity as an attribute to the edge
        graph[u][v]['capacity'] = capacity
        
        
# generate a directed er graph

num_nodes = [200, 500, 1000, 2000] # number of nodes for creating ER random graph
P = [0.7, 0.8, 0.9] # Probability for creating edges in ER random graph

for p in P:
    for n in num_nodes:
        
        # Generate an ER random graph same as nx.erdos_renyi_graph, but faster
        er_graph = nx.fast_gnp_random_graph(n, p, seed=2024, directed=False)
        
        # Generate capacity for this graph
        generate_random_capacities(er_graph)
        Capacity = {}
        for u, v in er_graph.edges():
            Capacity[(u, v)] = er_graph[u][v]['capacity']
        # save
        with open(root+'erdos_graph/Edge_C_' + str(n) + 'n_' + str(p) + 'p.pkl', 'wb') as f:
            pickle.dump(Capacity, f)
        print('Add capacity to one edge first:', len(Capacity.keys()))
        
        # use undirected erdos graph and add .to_directed (symmetric)
        G = er_graph.to_directed()
        print('Strongly connected:', nx.is_strongly_connected(G))
        print('# of nodes and edges, p:', G.number_of_nodes(), G.number_of_edges(), p)
        # nx.draw(G, with_labels=True, node_color='lightgreen', arrows=True)    
        # save
        nx.write_graphml(G, root+'erdos_graph/er_graph_' + str(n) + 'n_' + str(p) + 'p.graphml')
        print('----------------------------------------')

Add capacity to one edge first: 13762
Strongly connected: True
# of nodes and edges, p: 200 27524 0.7
----------------------------------------
Add capacity to one edge first: 87092
Strongly connected: True
# of nodes and edges, p: 500 174184 0.7
----------------------------------------
Add capacity to one edge first: 349390
Strongly connected: True
# of nodes and edges, p: 1000 698780 0.7
----------------------------------------
Add capacity to one edge first: 1399083
Strongly connected: True
# of nodes and edges, p: 2000 2798166 0.7
----------------------------------------
Add capacity to one edge first: 15777
Strongly connected: True
# of nodes and edges, p: 200 31554 0.8
----------------------------------------
Add capacity to one edge first: 99670
Strongly connected: True
# of nodes and edges, p: 500 199340 0.8
----------------------------------------
Add capacity to one edge first: 399706
Strongly connected: True
# of nodes and edges, p: 1000 799412 0.8
---------------------------

### generate k-shortest path

In [8]:
from itertools import islice
def k_shortest_paths(G, source, target, k, weight=None):
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))

### Read all graphs and their capacities and load as a group

In [7]:
#### select graphs ####
num_nodes = [200, 500, 1000, 2000] # number of nodes for creating ER random graph
P = [0.7, 0.8, 0.9] # Probability for creating edges in ER random graph
ran_group = []
ran_group_noC = []
for p in P:
    for n in num_nodes:
        G = nx.read_graphml(root+'erdos_graph/er_graph_' + str(n) + 'n_' + str(p) + 'p.graphml')
        ran_group_noC.append(G)
        print('Graph info', G.number_of_nodes(), G.number_of_edges())
        print('Connected:', nx.is_strongly_connected(G))
        with open(root+'erdos_graph/Edge_C_' + str(n) + 'n_' + str(p) + 'p.pkl', 'rb') as f:
            Edge_C = pickle.load(f)
        print('Len capacity keys:', len(Edge_C.keys()))
        g_test = nx.DiGraph(G)
        for u, v in G.edges():
            if (int(u), int(v)) in Edge_C.keys():
                g_test.add_edge(u, v, weight=Edge_C[(int(u), int(v))])
            else:
                g_test.add_edge(u, v, weight=Edge_C[(int(v), int(u))])
        ran_group.append(g_test)
        print('After adding Capacity info', g_test.number_of_nodes(), g_test.number_of_edges())
        print('Connected:', nx.is_strongly_connected(g_test))
        print('-------------------------------------')

Graph info 200 27524
Connected: True
Len capacity keys: 13762
After adding Capacity info 200 27524
Connected: True
-------------------------------------
Graph info 500 174184
Connected: True
Len capacity keys: 87092
After adding Capacity info 500 174184
Connected: True
-------------------------------------
Graph info 1000 698780
Connected: True
Len capacity keys: 349390
After adding Capacity info 1000 698780
Connected: True
-------------------------------------
Graph info 2000 2798166
Connected: True
Len capacity keys: 1399083
After adding Capacity info 2000 2798166
Connected: True
-------------------------------------
Graph info 200 31554
Connected: True
Len capacity keys: 15777
After adding Capacity info 200 31554
Connected: True
-------------------------------------
Graph info 500 199340
Connected: True
Len capacity keys: 99670
After adding Capacity info 500 199340
Connected: True
-------------------------------------
Graph info 1000 799412
Connected: True
Len capacity keys: 399706


## function define

In [8]:
# G: G(V, E, C)                           nx.weighted.graph
# STD: demands align with ST pairs        list[([s1, t1], dmd1), ([s2, t2], dmd2),...], (string, int)
# Pd: set of paths for every st pair      dict{[s1, t1]: [([path1], cost1), ([path2], cost2)...], [s2, t2]...}
# # of std pairs = # of keys in Pd
# k: k shortest path for every (s, t, d) tuple

def generate_reallocation(G, STD, Pd, k):
    
    # constraint 1
    A1 = []
    for i in range(len(STD)):
        a = np.zeros(len(STD)*k)
        a[k*i: k*i+k] = 1
        A1.append(a)
    A1 = np.array(A1)
    b1 = np.ones(len(STD))

    # constrain 2
    edges_list = list(G.edges())
    A2 = np.zeros((G.number_of_edges(), len(STD)*k))

    for i in range(len(STD)):
        paths = Pd[tuple(STD[i][0])] # possible paths
        for j in range(k):
            p = paths[j]   # path[j] is the path
            for n in range(len(p)-1):
                if (p[n], p[n+1]) in edges_list:
                    A2[edges_list.index((p[n], p[n+1]))][k*i+j] = STD[i][1]
                else:
                    continue  
    b2 = np.array(list(nx.get_edge_attributes(G,'weight').values()))
    zero_row_indices = np.where(A2.any(axis=1)==0)[0]
    A2 = np.delete(A2, zero_row_indices, axis=0)
    b2 = np.delete(b2, zero_row_indices, axis=0)

    for i in range(A2.shape[0]):
        A2[i] = A2[i]/b2[i]
        b2[i] = b2[i]/b2[i]
    
    # obj
    c = -1*np.concatenate([np.ones(k)*STD[i][1] for i in range(len(STD))])
        
    return A1, b1, A2, b2, c

## test dataset generation

In [12]:
### gen train ####
import time
warnings.filterwarnings("error")

random.seed(2024)
np.random.seed(2024)


pkg_idx = 0              # instance index for your data generation
success_cnt = 0
fail_cnt = 0
bounds = (0., 1.)

max_iter = 15000
num = 1                  # number of instance generated

k = 4                    # k-shortest path
max_d = 5000             # demand max value
min_d = 1000             # demand min value

number_of_st = 10        # number of st pairs

graph_info = []
for g in range(len(ran_group)):
    stds = []
    ips = []
    success_cnt = 0
    times = []
    for n in range(num+200): # in case failsure case
        
        # generate st pairs with demand value 
        std = []
        Pd = {}
        count_std = 0
        while count_std != number_of_st:
            st = np.random.choice(ran_group[g].nodes(), 2, replace=False)
            d = random.uniform(min_d, max_d)
            k_paths = k_shortest_paths(ran_group_noC[g], st[0], st[1], k=k)
            if len(k_paths) != k:
                continue
            else:
                Pd[(st[0], st[1])] = k_paths
                std.append((st, d))
                count_std += 1

        A1, b1, A2, b2, c = generate_reallocation(ran_group[g], std, Pd, k)
        A = np.vstack([A1, A2])
        b = np.hstack([b1, b2])
        
        n_time = time.time()
        try:
            A_eq = None
            b_eq = None
            A_ub = A
            b_ub = b
            res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, bounds=bounds, 
                          method='interior-point')
            times.append(time.time()-n_time)
            print(res)
        except (LinAlgWarning, OptimizeWarning, AssertionError):
            fail_cnt += 1
            continue
        else:
            if res.success and not np.isnan(res.fun):
                ips.append((torch.from_numpy(A).to(torch.float), torch.from_numpy(b).to(torch.float), torch.from_numpy(c).to(torch.float)))
                success_cnt += 1
                stds.append(std)
        if success_cnt == num:
            break

    with open(root+'/raw/instance_'+str(pkg_idx)+'_stds.pkl','wb') as f:
        pickle.dump(stds, f)
    with gzip.open(f'{root}/raw/instance_{pkg_idx}.pkl.gz', "wb") as file:
        pickle.dump(ips, file)
    pkg_idx += 1
#     print('-----------------------------------------')
#     print('Graph finished info:', ran_group[g].number_of_nodes(), ran_group[g].number_of_edges())
#     print('Average time used:', sum(times)/len(times))
#     print('-----------------------------------------')
    graph_info.append((ran_group[g].number_of_nodes(), ran_group[g].number_of_edges(), sum(times)/len(times)))

np.save(root+'/raw/erdos_test_'+str(number_of_st)+'st_info', graph_info)
for i in graph_info:
    print('Graph info and average time used:', i)

    
    
warnings.resetwarnings()

      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -31349.513682428136
            x: [ 2.109e-01  2.156e-01 ...  2.166e-01  1.963e-01]
          nit: 7
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -35319.265433685934
            x: [ 2.479e-01  1.630e-01 ...  1.652e-01  2.416e-01]
          nit: 7
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -25084.20758680875
            x: [ 2.199e-01  2.097e-01 ...  2.067e-01  1.909e-01]
          nit: 7
 intermediate: []
      message: Optimization terminated successfully.
      success: True
       status: 0
          fun: -27989.14581541008
            x: [ 2.184e-01  1.853e-01 ...  1.904e-01  1.925e-01]
          nit: 7
 intermediate: []
Graph info and average time used: (200, 35662, 0.004581928253173828)
Graph info and average time used: